각 test_ID에 교통사고 값(ECLO)을 예측하기 

ECLO = 사망자수 * 10 + 중상자수 * 5 + 경상자수 * 3 + 부상자수 * 1

- 출퇴근 시간, 새벽 시간(음주, 졸음)에 많이 발생?

- 요일에 따라 달라진다? -> 인구 밀집

- 날씨 영향?

- 사고가 빈번한 지역?

- cctv가 없거나 신호 시간이 짧거나

- 도로 상태, 형태

- 노인이 많으면 사고가 자주 발생

# 라이브러리 불러오기

In [30]:
import pandas as pd


# 데이터 불러오기

In [148]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

train_df.head(3)
train_df.shape

test_df.head(3)
test_df.shape

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,상해없음,보행자,여,70세,중상,0,1,0,0,5
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,상해없음,보행자,남,61세,경상,0,0,1,0,3
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,상해없음,보행자,남,38세,경상,0,0,1,0,3


(39609, 23)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차


(10963, 8)

In [15]:
# 추가할 수 있는 data

cctv_df = pd.read_csv('./data/external_open/대구 CCTV 정보.csv', encoding = 'cp949')
security_light_df = pd.read_csv('./data/external_open/대구 보안등 정보.csv', encoding = 'cp949', low_memory = False)
child_protection_df = pd.read_csv('./data/external_open/대구 어린이 보호 구역 정보.csv', encoding = 'cp949')
parking_df = pd.read_csv('./data/external_open/대구 주차장 정보.csv', encoding = 'cp949')

# cctv_df.head(2)
# security_light_df.head(2)
# child_protection_df.head(2)
# parking_df.head(2)

# EDA

## 결측치 확인

In [32]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39609 entries, 0 to 39608
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           39609 non-null  object
 1   사고일시         39609 non-null  object
 2   요일           39609 non-null  object
 3   기상상태         39609 non-null  object
 4   시군구          39609 non-null  object
 5   도로형태         39609 non-null  object
 6   노면상태         39609 non-null  object
 7   사고유형         39609 non-null  object
 8   사고유형 - 세부분류  39609 non-null  object
 9   법규위반         39609 non-null  object
 10  가해운전자 차종     39609 non-null  object
 11  가해운전자 성별     39609 non-null  object
 12  가해운전자 연령     39609 non-null  object
 13  가해운전자 상해정도   39609 non-null  object
 14  피해운전자 차종     38618 non-null  object
 15  피해운전자 성별     38618 non-null  object
 16  피해운전자 연령     38618 non-null  object
 17  피해운전자 상해정도   38618 non-null  object
 18  사망자수         39609 non-null  int64 
 19  중상자수         39609 non-nu

=> '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'에 약 1000개의 결측치 존재

In [18]:
test_df.info() # 결측치 없음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10963 entries, 0 to 10962
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10963 non-null  object
 1   사고일시    10963 non-null  object
 2   요일      10963 non-null  object
 3   기상상태    10963 non-null  object
 4   시군구     10963 non-null  object
 5   도로형태    10963 non-null  object
 6   노면상태    10963 non-null  object
 7   사고유형    10963 non-null  object
dtypes: object(8)
memory usage: 685.3+ KB


=> '피해운전자 차종', '피해운전자 성별', '피해운전자 연령', '피해운전자 상해정도'에 결측치가 존재하는 이유 -> '사고 유형'이 '챠랑단독'이기 때문이다

=> 이런 결측치는 어떻게 처리를 해야하나

## 시군구, 날짜, 도로형태 분리하기

In [149]:
# 시군구 나누는 함수 구현

def si_gun_gu(df):
    lst = df['시군구'].str.split(' ')
    df['시'] = lst.str[0]
    df['군'] = lst.str[1]
    df['구'] = lst.str[2]
    return df

In [150]:
train_df = si_gun_gu(train_df)
test_df = si_gun_gu(test_df)

train_df.head(3)
test_df.head(3)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,피해운전자 연령,피해운전자 상해정도,사망자수,중상자수,경상자수,부상자수,ECLO,시,군,구
0,ACCIDENT_00000,2019-01-01 00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,70세,중상,0,1,0,0,5,대구광역시,중구,대신동
1,ACCIDENT_00001,2019-01-01 00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,61세,경상,0,0,1,0,3,대구광역시,달서구,감삼동
2,ACCIDENT_00002,2019-01-01 01,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,38세,경상,0,0,1,0,3,대구광역시,수성구,두산동


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,시,군,구
0,ACCIDENT_39609,2022-01-01 01,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,대구광역시,수성구,상동
1,ACCIDENT_39610,2022-01-01 01,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,대구광역시,수성구,지산동
2,ACCIDENT_39611,2022-01-01 04,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,대구광역시,수성구,수성동2가


In [151]:
# 사고일시를 나누는 함수 

def year_mon_day(df):
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    df['year'] = df['사고일시'].dt.year
    df['month'] = df['사고일시'].dt.month
    df['day'] =  df['사고일시'].dt.day
    df['time'] = df['사고일시'].dt.hour
    return df

In [152]:
train_df = year_mon_day(train_df)
test_df = year_mon_day(test_df)

train_df.head(3)
test_df.head(3)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,경상자수,부상자수,ECLO,시,군,구,year,month,day,time
0,ACCIDENT_00000,2019-01-01 00:00:00,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,0,0,5,대구광역시,중구,대신동,2019,1,1,0
1,ACCIDENT_00001,2019-01-01 00:00:00,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,1,0,3,대구광역시,달서구,감삼동,2019,1,1,0
2,ACCIDENT_00002,2019-01-01 01:00:00,화요일,맑음,대구광역시 수성구 두산동,단일로 - 기타,건조,차대사람,차도통행중,안전운전불이행,...,1,0,3,대구광역시,수성구,두산동,2019,1,1,1


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,시,군,구,year,month,day,time
0,ACCIDENT_39609,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,대구광역시,수성구,상동,2022,1,1,1
1,ACCIDENT_39610,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,대구광역시,수성구,지산동,2022,1,1,1
2,ACCIDENT_39611,2022-01-01 04:00:00,토요일,맑음,대구광역시 수성구 수성동2가,교차로 - 교차로안,건조,차대차,대구광역시,수성구,수성동2가,2022,1,1,4


In [153]:
# 도로형태를 '-'를 기준으로 분리하는 함수 구현
def road(df):
    lst = df['도로형태'].str.split(' - ')
    df['도로형태 - 대분류'] = lst.str[0]
    df['도로형태 - 소분류'] = lst.str[1]
    return df

In [154]:
train_df = road(train_df)
test_df = road(test_df)

train_df.head(2)
test_df.head(2)

,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,...,ECLO,시,군,구,year,month,day,time,도로형태 - 대분류,도로형태 - 소분류
0,ACCIDENT_00000,2019-01-01,화요일,맑음,대구광역시 중구 대신동,단일로 - 기타,건조,차대사람,길가장자리구역통행중,안전운전불이행,...,5,대구광역시,중구,대신동,2019,1,1,0,단일로,기타
1,ACCIDENT_00001,2019-01-01,화요일,흐림,대구광역시 달서구 감삼동,단일로 - 기타,건조,차대사람,보도통행중,기타,...,3,대구광역시,달서구,감삼동,2019,1,1,0,단일로,기타


,ID,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,시,군,구,year,month,day,time,도로형태 - 대분류,도로형태 - 소분류
0,ACCIDENT_39609,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 상동,교차로 - 교차로안,건조,차대사람,대구광역시,수성구,상동,2022,1,1,1,교차로,교차로안
1,ACCIDENT_39610,2022-01-01 01:00:00,토요일,맑음,대구광역시 수성구 지산동,단일로 - 기타,건조,차대사람,대구광역시,수성구,지산동,2022,1,1,1,단일로,기타


## 상관관계 확인

# 데이터 전처리

## train에 있고 test에 없는 값 삭제

In [89]:
train_df['기상상태'].unique()
train_df['도로형태'].unique()
train_df['노면상태'].unique()
train_df['사고유형'].unique()
train_df['사고유형 - 세부분류'].unique()
train_df['법규위반'].unique()

array(['맑음', '흐림', '기타', '비', '안개', '눈'], dtype=object)

array(['단일로 - 기타', '교차로 - 교차로안', '기타 - 기타', '단일로 - 터널', '단일로 - 지하차도(도로)내',
       '단일로 - 교량', '교차로 - 교차로횡단보도내', '주차장 - 주차장', '교차로 - 교차로부근',
       '단일로 - 고가도로위', '미분류 - 미분류'], dtype=object)

array(['건조', '젖음/습기', '서리/결빙', '기타', '침수', '적설'], dtype=object)

array(['차대사람', '차대차', '차량단독'], dtype=object)

array(['길가장자리구역통행중', '보도통행중', '차도통행중', '추돌', '측면충돌', '기타', '공작물충돌', '횡단중',
       '정면충돌', '후진중충돌', '전도전복 - 전도', '도로외이탈 - 추락', '도로외이탈 - 기타',
       '전도전복 - 전복'], dtype=object)

array(['안전운전불이행', '기타', '교차로운행방법위반', '신호위반', '중앙선침범', '안전거리미확보', '차로위반',
       '보행자보호의무위반', '직진우회전진행방해', '과속', '불법유턴'], dtype=object)

In [91]:
test_df['기상상태'].unique()
test_df['도로형태'].unique()
test_df['노면상태'].unique()
test_df['사고유형'].unique()

array(['맑음', '흐림', '기타', '비', '눈'], dtype=object)

array(['교차로 - 교차로안', '단일로 - 기타', '교차로 - 교차로횡단보도내', '교차로 - 교차로부근',
       '단일로 - 지하차도(도로)내', '기타 - 기타', '단일로 - 교량', '단일로 - 고가도로위',
       '주차장 - 주차장', '단일로 - 터널', '미분류 - 미분류'], dtype=object)

array(['건조', '젖음/습기', '서리/결빙', '기타', '침수', '적설'], dtype=object)

array(['차대사람', '차대차', '차량단독'], dtype=object)

=> train '기상상태'에는 안개가 있고 test '기상상태'에는 안개가 없어 삭제한다

In [155]:
# '기상상태'에서 '안개' 삭제
train_df = train_df[train_df['기상상태'] != '안개']

train_df['기상상태'].unique()

array(['맑음', '흐림', '기타', '비', '눈'], dtype=object)

## 연령을 수치로

In [156]:
train_df['가해운전자 연령'] = train_df['가해운전자 연령'].str.replace('세', '')
# train_df['가해운전자 연령'] = train_df['가해운전자 연령'].astype('int')
# train_df['가해운전자 연령']

In [157]:
train_df['가해운전자 연령'].unique()

array(['51', '39', '70', '49', '30', '52', '61', '68', '66', '62', '미분류',
       '56', '64', '46', '74', '28', '72', '33', '27', '50', '19', '41',
       '54', '76', '35', '25', '60', '85', '20', '21', '67', '59', '40',
       '65', '47', '48', '43', '58', '31', '44', '53', '32', '37', '22',
       '24', '34', '26', '75', '23', '38', '18', '45', '36', '57', '71',
       '69', '29', '17', '42', '88', '79', '55', '63', '78', '73', '89',
       '16', '10', '80', '13', '82', '14', '86', '83', '77', '84', '15',
       '81', '11', '9', '87', '4', '8', '12', '90 이상', '7', '98 이상', '90',
       '5'], dtype=object)

=> 가해운전자 연량에 미분류(900개), 18, 17, 16, 10, 13, 14, 15, 11, 9, 4, 8, 12, ,7, 5는 흠,,

=> 90 이상, 98 이상, 90은 그냥 90이상으로 변경

In [144]:
train_df[train_df['가해운전자 연령'] == '5'].T
train_df[train_df['가해운전자 연령'] == '미분류'].iloc[1].T

,29398
ID,ACCIDENT_29398
사고일시,2021-03-04 15:00:00
요일,목요일
기상상태,맑음
시군구,대구광역시 서구 중리동
도로형태,기타 - 기타
노면상태,건조
사고유형,차대사람
사고유형 - 세부분류,기타
법규위반,기타


ID                  ACCIDENT_00038
사고일시           2019-01-02 15:00:00
요일                             수요일
기상상태                            맑음
시군구                   대구광역시 서구 이현동
도로형태                      단일로 - 기타
노면상태                            건조
사고유형                           차대차
사고유형 - 세부분류                   측면충돌
법규위반                       안전거리미확보
가해운전자 차종                      기타불명
가해운전자 성별                      기타불명
가해운전자 연령                       미분류
가해운전자 상해정도                    기타불명
피해운전자 차종                        승용
피해운전자 성별                         남
피해운전자 연령                       32세
피해운전자 상해정도                    부상신고
사망자수                             0
중상자수                             0
경상자수                             0
부상자수                             1
ECLO                             1
시                            대구광역시
군                               서구
구                              이현동
year                          2019
month                            1
day                 

In [142]:
train_df[train_df['가해운전자 연령'] == '미분류'].loc[:, '사고일시' : '가해운전자 상해정도']

,사고일시,요일,기상상태,시군구,도로형태,노면상태,사고유형,사고유형 - 세부분류,법규위반,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도
11,2019-01-01 14:00:00,화요일,맑음,대구광역시 동구 신기동,교차로 - 교차로안,건조,차대차,기타,안전운전불이행,기타불명,기타불명,미분류,기타불명
38,2019-01-02 15:00:00,수요일,맑음,대구광역시 서구 이현동,단일로 - 기타,건조,차대차,측면충돌,안전거리미확보,기타불명,기타불명,미분류,기타불명
83,2019-01-03 20:00:00,목요일,맑음,대구광역시 북구 태전동,단일로 - 기타,건조,차대차,측면충돌,안전운전불이행,기타불명,남,미분류,기타불명
145,2019-01-05 17:00:00,토요일,맑음,대구광역시 북구 산격동,단일로 - 기타,건조,차대사람,기타,안전운전불이행,기타불명,기타불명,미분류,기타불명
186,2019-01-07 02:00:00,월요일,맑음,대구광역시 달서구 진천동,단일로 - 기타,건조,차대차,기타,안전운전불이행,기타불명,기타불명,미분류,기타불명
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39486,2021-12-28 13:00:00,화요일,맑음,대구광역시 달서구 진천동,단일로 - 기타,건조,차대사람,기타,안전운전불이행,기타불명,기타불명,미분류,기타불명
39518,2021-12-29 16:00:00,수요일,맑음,대구광역시 수성구 범어동,단일로 - 기타,건조,차대사람,기타,안전운전불이행,기타불명,기타불명,미분류,기타불명
39540,2021-12-30 11:00:00,목요일,맑음,대구광역시 수성구 범어동,단일로 - 기타,건조,차대차,기타,안전운전불이행,기타불명,기타불명,미분류,기타불명
39578,2021-12-31 12:00:00,금요일,맑음,대구광역시 달성군 구지면,단일로 - 기타,건조,차대차,기타,안전거리미확보,기타불명,기타불명,미분류,기타불명
